<a href="https://colab.research.google.com/github/aaronfarquhar/13006610-dataanylitics/blob/main/Assessment_2_DAOTW_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
#imports for python
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

#read data source and create dataframe
df = pd.read_csv('https://raw.githubusercontent.com/aaronfarquhar/13006610-dataanylitics/main/Collated_Data_Assessment_2_13006610.csv')

In [2]:
print(tf.__version__) #for checking the tensorflow import

2.12.0


In [3]:
#check data is present and succsessfully read
print(df)

      temp  visib  day  mo  year  NUM_COLLISIONS
0     76.0    8.9    1   7  2012             505
1     78.5   10.0    1   7  2012             533
2     79.0    9.2    1   7  2012             528
3     73.1   10.0    1   7  2012             530
4     80.3   10.0    1   7  2012             564
...    ...    ...  ...  ..   ...             ...
3160  51.5    6.3    6  12  2020             278
3161  38.5   10.0    7  12  2020             194
3162  39.3    9.8    7  12  2020             241
3163  53.4    6.3    7  12  2020             265
3164  34.2   10.0    7  12  2020             184

[3165 rows x 6 columns]


the collated data will be the same as before

In [4]:
#temp and collisions table
inputs_for_training = [df['visib'], df['NUM_COLLISIONS']]
header = ['Visibility', 'Number of Collisions']
combined1 = pd.concat(inputs_for_training, axis = 1, keys=header)
print (combined1)

      Visibility  Number of Collisions
0            8.9                   505
1           10.0                   533
2            9.2                   528
3           10.0                   530
4           10.0                   564
...          ...                   ...
3160         6.3                   278
3161        10.0                   194
3162         9.8                   241
3163         6.3                   265
3164        10.0                   184

[3165 rows x 2 columns]


In [5]:
#training and testing data
train_dataset = combined1.sample(frac=0.8, random_state=0)
test_dataset = combined1.drop(train_dataset.index)

#prints for checking
print(train_dataset)
print(test_dataset)


      Visibility  Number of Collisions
2533         6.3                   592
1436         6.4                   638
2567        10.0                   722
3139         9.3                   320
1473         9.7                   742
...          ...                   ...
1181         6.9                   545
337          2.0                   707
46          10.0                   618
747          9.8                   451
2891         9.9                   127

[2532 rows x 2 columns]
      Visibility  Number of Collisions
0            8.9                   505
3           10.0                   530
7            8.1                   574
21          10.0                   638
24           9.5                   490
...          ...                   ...
3142        10.0                   315
3143        10.0                   238
3145        10.0                   360
3153        10.0                   348
3159         8.6                   285

[633 rows x 2 columns]


In [6]:
train_copy = train_dataset.copy()
test_copy = test_dataset.copy()

training_label = train_copy.pop('Number of Collisions')
testing_labels = test_copy.pop('Number of Collisions')

print(test_copy)
print(testing_labels)

      Visibility
0            8.9
3           10.0
7            8.1
21          10.0
24           9.5
...          ...
3142        10.0
3143        10.0
3145        10.0
3153        10.0
3159         8.6

[633 rows x 1 columns]
0       505
3       530
7       574
21      638
24      490
       ... 
3142    315
3143    238
3145    360
3153    348
3159    285
Name: Number of Collisions, Length: 633, dtype: int64


In [7]:
from keras.layers.preprocessing import normalization
normalisation_layer = tf.keras.layers.Normalization(input_shape=[1,], axis = None)
#convert values to -1 to 1
normalisation_layer.adapt(np.array(train_copy))

The sateps above are the same as for the linear regresion model in part one and do not need elaberation. how ever below a number of extra layers are created between the normalisation layer and the output layer. theses extra, dense layers start at 256 nodes and half the number of nodes at each layer. this should hopefully increase the accuracy of the model though a Deep Neural Networks value in so simple a case is dubious.

In [8]:
model=tf.keras.Sequential([
    normalisation_layer,
    tf.keras.layers.Dense(256, activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(128, activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(64, activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(32, activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(16, activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(8, activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(4, activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(2, activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(units=1)
])

In [9]:
model.compile(
    optimizer = tf.optimizers.Adam(learning_rate = 0.1),
    loss = 'mean_absolute_error'
    )

In [10]:
%%time
model_output = model.fit(train_copy, training_label, epochs=100, validation_split = 0.2, verbose = 1 )

Epoch 1/100
64/64 [==============================] - 3s 9ms/step - loss: 558.0606 - val_loss: 554.5764
Epoch 2/100
64/64 [==============================] - 0s 7ms/step - loss: 551.6591 - val_loss: 548.1765
Epoch 3/100
64/64 [==============================] - 0s 7ms/step - loss: 545.2592 - val_loss: 541.7766
Epoch 4/100
64/64 [==============================] - 0s 7ms/step - loss: 538.8592 - val_loss: 535.3766
Epoch 5/100
64/64 [==============================] - 0s 6ms/step - loss: 532.4594 - val_loss: 528.9767
Epoch 6/100
64/64 [==============================] - 0s 6ms/step - loss: 526.0594 - val_loss: 522.5768
Epoch 7/100
64/64 [==============================] - 0s 7ms/step - loss: 519.6595 - val_loss: 516.1769
Epoch 8/100
64/64 [==============================] - 0s 5ms/step - loss: 513.2598 - val_loss: 509.7771
Epoch 9/100
64/64 [==============================] - 0s 5ms/step - loss: 506.8598 - val_loss: 503.3772
Epoch 10/100
64/64 [==============================] - 0s 6ms/step - loss:

the model progresses with a final loss of roughly 105 on the final epoch. It's accuracy is shown below and is on par with the previous Linear Regresion model. 

In [11]:
accuracy = model.evaluate(test_copy, testing_labels)

20/20 [==============================] - 0s 2ms/step - loss: 97.5620


In [12]:
print(accuracy)

97.56195068359375


In [13]:
new_data = np.array([6,3.5,10.0,1])

predictions = model.predict(new_data)
print(predictions)

1/1 [==============================] - 0s 165ms/step
[[553.36896]
 [553.36896]
 [553.36896]
 [553.36896]]


**Conclutions**

from this report it can be concluded that in certain simpler cases, a liear regrestion model can be more suitable for a situation then a DNN, as the LR model seemed to work better.